#### Name: `Jike Lu`
#### AndrewID: `jikelu`

## Determining the most popular gender neutral names

The United States Social Security administration keeps records of all births and provides some of this [data](https://www.ssa.gov/oact/babynames) to the public in a file where each line is of the format:

KY,F,1912,Dorothy,209

this is to be interpreted as "In 1912, 209 female babies were born in Kentucky who were given the first name Dorothy".

In this exercise you are to write a pyspark program that works with RDDs to determine the most popular gender neutral names.  We define a gender neutral name as a baby name that has been given to both a boy and girl baby.  We define a popular gender neutral name as a name where the ratio of the number of boys with that name to the number of girls with that name is in the range \[0.25..4\]

The whole babynames file from the SSA has 6028151 records (lines) and information on ??? babies.   To facilitate development, I've sampled 100,000 lines in the file `babynames-100k.csv`.  During development, working with the sample.  Once done, set `test` to `False` run your code and submit your notebook

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

tar: spark-3.3.2-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [2]:
test = False
if test:
    file_name = 'babynames-100k.csv'
else:
    file_name = 'babynames2018_state_gender_year_fname_number.csv'

In [3]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## Babies

In [5]:
!head babynames2018_state_gender_year_fname_number.csv

AK,F,1910,Mary,14
AK,F,1910,Annie,12
AK,F,1910,Anna,10
AK,F,1910,Margaret,8
AK,F,1910,Helen,7
AK,F,1910,Elsie,6
AK,F,1910,Lucy,6
AK,F,1910,Dorothy,5
AK,F,1911,Mary,12
AK,F,1911,Margaret,7


In [6]:
from pyspark.sql.types import *

In [7]:
schema = StructType([
    StructField('state', StringType(), False),
    StructField('gender', StringType(), False),
    StructField('year', IntegerType(), False),
    StructField('name', StringType(), False),
    StructField('number', IntegerType(), False)
])

In [8]:
baby_names = spark.read.csv(file_name, schema=schema)

In [9]:
type(baby_names)

pyspark.sql.dataframe.DataFrame

In [10]:
baby_names.createOrReplaceTempView("baby_names")

In [11]:
spark.sql('''
SELECT sum(number) as cnt
  FROM baby_names
''').show()

+---------+
|      cnt|
+---------+
|311155210|
+---------+



In [12]:
male_names = spark.sql('''
SELECT name, SUM(number) AS male_count
FROM baby_names
WHERE gender = 'M'
GROUP BY name
''')

female_names = spark.sql('''
SELECT name, SUM(number) AS female_count
FROM baby_names
WHERE gender = 'F'
GROUP BY name
''')

In [ ]:
male_names.count(), female_names.count()

(13785, 20852)

In [13]:
male_names.createOrReplaceTempView("male_names")
female_names.createOrReplaceTempView("female_names")

In [14]:
both_names = spark.sql('''
SELECT
    m.name,
    m.male_count,
    f.female_count,
    (m.male_count + f.female_count) AS total_count
FROM male_names m
JOIN female_names f
ON m.name = f.name
''')

In [15]:
both_names.count()

3042

In [16]:
both_names.createOrReplaceTempView("both_names")

In [17]:
spark.sql('''
SELECT
    name,
    male_count,
    female_count,
    total_count
FROM both_names
ORDER BY total_count DESC
LIMIT 10

''').show()

+-------+----------+------------+-----------+
|   name|male_count|female_count|total_count|
+-------+----------+------------+-----------+
|  James|   4997327|       18257|    5015584|
|   John|   4869607|       15677|    4885284|
| Robert|   4734038|       15116|    4749154|
|Michael|   4349307|       17217|    4366524|
|William|   3890923|       10211|    3901134|
|   Mary|      9642|     3741196|    3750838|
|  David|   3597725|        8185|    3605910|
|Richard|   2539873|        4994|    2544867|
| Joseph|   2522812|        5625|    2528437|
|Charles|   2273068|        7532|    2280600|
+-------+----------+------------+-----------+

